In [1]:
!pip install lightgbm

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
from sklearn.metrics import accuracy_score, f1_score, log_loss
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
df = pd.read_csv("final_features.csv")

print(df.shape)
df.head()

(90000, 244)


,id,is_duplicate,q1_len_x,q2_len_x,q1_num_words_x,q2_num_words_x,word_common_x,word_total_x,word_share_x,cwc_min,...,86_y,87_y,88_y,89_y,90_y,91_y,92_y,93_y,94_y,95_y
0,0,1,75,76,13,13,12,26,0.46,0.874989,...,24.551529,-7.684039,-24.883157,6.526309,-24.260105,24.287874,23.207100,21.139578,-18.450893,36.307079
1,1,0,48,56,13,16,8,24,0.33,0.666644,...,24.103431,-11.546866,-62.848033,15.441650,-9.592031,16.754833,26.424757,28.875602,-23.002500,16.389976
2,2,0,104,119,28,21,4,38,0.11,0.000000,...,39.676087,0.289840,-21.327893,31.244260,-15.868776,10.563364,-9.109160,19.774184,-11.013100,28.307958
3,3,0,58,145,14,32,1,34,0.03,0.000000,...,11.053748,-5.903925,0.677170,-7.418554,-5.314642,-12.190648,-3.195340,8.207225,6.415468,9.979167
4,4,0,34,49,5,9,3,13,0.23,0.749981,...,7.919799,-5.115303,-11.169791,-4.767877,-16.489244,13.219784,0.407731,17.408289,-16.857232,22.797968


In [4]:
# Target
y = df['is_duplicate']  

# Features
X = df.drop(columns=['id','is_duplicate'], errors='ignore')


In [5]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42, stratify=y)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, log_loss

rf = RandomForestClassifier(random_state=42, n_jobs=-1)

param_grid = {
    "n_estimators": [200, 300, 500],
    "max_depth": [10, 15, 20, None],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ["sqrt", "log2", None]
}

# Use log_loss as scorer (minimize it → better probability calibration)
grid = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    scoring="neg_log_loss",  # sklearn wants a score → negative log loss
    cv=3,
    verbose=2,
    n_jobs=-1
)

grid.fit(X_train, y_train)

print("Best Params:", grid.best_params_)
print("Best Log Loss:", -grid.best_score_)

Fitting 3 folds for each of 324 candidates, totalling 972 fits


In [ ]:
from sklearn.calibration import CalibratedClassifierCV

# Try isotonic calibration (works best if you have enough data)
calibrated_rf = CalibratedClassifierCV(base_estimator=rf, method='isotonic', cv=5)
calibrated_rf.fit(X_train, y_train)

y_pred_cal = calibrated_rf.predict(X_test)
y_pred_proba_cal = calibrated_rf.predict_proba(X_test)

print("\nAfter Calibration:")
print("Accuracy:", accuracy_score(y_test, y_pred_cal))
print("F1 Score:", f1_score(y_test, y_pred_cal))
print("Log Loss:", log_loss(y_test, y_pred_proba_cal))


In [ ]:
from sklearn.calibration import calibration_curve
import matplotlib.pyplot as plt

# Before calibration
prob_true, prob_pred = calibration_curve(y_test, y_pred_proba[:, 1], n_bins=10)
# After calibration
prob_true_cal, prob_pred_cal = calibration_curve(y_test, y_pred_proba_cal[:, 1], n_bins=10)

plt.figure(figsize=(7,7))
plt.plot(prob_pred, prob_true, "o-", label="Before Calibration")
plt.plot(prob_pred_cal, prob_true_cal, "o-", label="After Calibration")
plt.plot([0,1],[0,1], "--", color="gray", label="Perfect Calibration")

plt.xlabel("Predicted probability")
plt.ylabel("True probability")
plt.title("Calibration Curve")
plt.legend()
plt.show()


In [ ]:
xgb = XGBClassifier(
    n_estimators=300,
    learning_rate=0.1,
    max_depth=8,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1,
    eval_metric="logloss" 
)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
y_pred_proba = xgb.predict_proba(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("Log Loss:", log_loss(y_test, y_pred_proba))

In [ ]:
lr = LogisticRegression(max_iter=2000, n_jobs=-1, solver="saga")
lr.fit(X_train, y_train)

y_pred = lr.predict(X_test)
y_pred_proba = lr.predict_proba(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("Log Loss:", log_loss(y_test, y_pred_proba))


C:\Users\SANHITH REDDY\.conda\envs\quora_env\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
lgb_model = lgb.LGBMClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=-1,
    num_leaves=64,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)

lgb_model.fit(X_train, y_train)

y_pred = lgb_model.predict(X_test)
y_pred_proba = lgb_model.predict_proba(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("Log Loss:", log_loss(y_test, y_pred_proba))
